## Part 1: 搭建一个分词工具

### Part 1.1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [1]:
# TODO: 第一步： 从dic.txt中读取所有中文词。
dic_file = open('./dic.txt', 'r', encoding='utf-8')
content = dic_file.read()
dic_file.close()
#  hint: 思考一下用什么数据结构来存储这个词典会比较好？ 要考虑我们每次查询一个单词的效率。 
dic_words = []
for ele in content.split(','):
    dic_words.append(ele)

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

print (sum(word_prob.values()))
print(dic_words)

1.0000000000000002
['北京', '的', '天', '气', '天气', '真', '好', '真好', '啊', '真好啊', '今', '今天', '课程', '内容', '有', '很', '很有', '意思', '有意思', '课', '程', '经常', '意见', '意', '见', '有意见', '分歧', '分', '歧']


In [2]:
#  分数（10）
## TODO 请编写word_segment_naive函数来实现对输入字符串的分词
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    # 注意 是基于枚举的方法，不考虑正向最大匹配
    # max_len = 0 # 获取最大分词长度
    # for w in dic_words:
    #     if len(w) > max_len:
    #         max_len = len(w)
    # 
    # TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。
    # 定义一个空列表来储存分词结果
    # result_word = []
    # k = 0 # 切割输入字符串从下标0开始
    # 
    # while k < len(input_str):
    #     test_str = input_str[k: max_len+k]
    #     if test_str in dic_words:
    #         result_word.append(test_str)
    #         k = max_len + k
    #     else:
    #         while len(test_str) > 1:
    #             test_str = test_str[0:len(test_str)-1]
    #             if test_str in dic_words or len(test_str) == 1:
    #                 result_word.append(test_str)
    #                 k = len(test_str) - 1 + k
    #             k = len(test_str) - 1
    # return result_word
                        
    """重新考虑基于枚举的方法, 首先要求所有的分词结果，就是求该句话在字典下的全切分。
    """
    # 定义一个空间暂时储存排列
    mem = [[] for _ in range(len(input_str))]
    
    for i in range(0, len(input_str)):
        for j in range(0, i+1): # 从一个字节开始，考虑每一个词是否在字典中存在
            if j == 0: # 如果刚刚取出的字符串就在字典中则加入mem
                if input_str[j:i+1] in dic_words:
                    mem[i].append([input_str[j:i+1]])
                continue # 继续循环，减少字符串长度看在不在字典中
            
            # 当取出的字符串不在其中时，就要根据前一个状态往下推，要确定依赖的前状态存在且达成转移条件
            if mem[j-1] and input_str[j:i+1] in dic_words: 
                for status in mem[j-1]: # 对应前状态的每一个可能加上现在新出现的词
                    mem[i].append(status + [input_str[j:i+1]])
    return mem[-1]
        
    
    
        
    
    # segments = []  # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
                   # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]

    # TODO: 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
    # best_segment = 
    # best_score = 
    # for seg in segments:
    #     # TODO ...
    
     

In [3]:
word_segment_naive('今天天气好')



[['今天', '天气', '好'],
 ['今', '天', '天气', '好'],
 ['今天', '天', '气', '好'],
 ['今', '天', '天', '气', '好']]